## Segmenting and Clustering Neighborhoods in Toronto

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Load and Explore Data

In [2]:
tables=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
print(tables[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

table[0] is the desired data set.

In [4]:
df=tables[0]
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [5]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning

Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

In [6]:
df=df[df['Neighbourhood']!='Not assigned']
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."



Reset the index to start from 0.

In [7]:
df.reset_index(inplace=True,drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df.shape

(103, 3)

#### Get Coordinate of Each Postal Code

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [9]:
import geocoder

In [10]:
def get_coords(postal_code):
    lat_lng_coords=None
    while lat_lng_coords is None :
        g=geocoder.google(f"{postal_code}, Toronto, Ontario")
        lat_lng_coords=g.latlng
        print(g)
    return lat_lng_coords


In [11]:
df_coords=pd.DataFrame(columns=['Latitude','Longitude'])
df_coords

,Latitude,Longitude


In [12]:
for postal_code in df['Postal Code']:
    coords=get_coords(postal_code)
    df_coords.append([coords[0],coords[1]])
df_coords

<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>


KeyboardInterrupt: 

Failed to get coordinates using geocoder. Hence using csv file to get coordinates.

In [13]:
df_coords=pd.read_csv("https://cocl.us/Geospatial_data")
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Inner Joining **df** and **df_coords** using **merge()**.

In [14]:
df=pd.merge(df,df_coords,how='inner')

In [15]:
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [17]:
df_toronto=df[df['Borough'].str.contains("Toronto")]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
df_toronto.shape

(39, 5)

#### Get latitude and longitude values of Toronto

In [19]:
# import libraries
from geopy.geocoders import Nominatim
import folium
import requests

In [20]:
address='Toronto, Canada'
geolocator=Nominatim(user_agent='totronto_explorer')
location=geolocator.geocode(address)
lat=location.latitude
lng=location.longitude
print(f"Geographical coordinate of Toronto {lat},{lng}")

Geographical coordinate of Toronto 43.6534817,-79.3839347


#### Create Map of Toronto with Neighborhood 

In [21]:
map_toronto=folium.Map(location=[lat,lng],zoom_start=10)
for lt, lg, borough, neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighbourhood']):
    label=f"{neighbourhood}, {borough}"
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lt,lg],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto

### Define FourSquare Credentials

#### Function to Get All Nearby Venues 

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
df_venues=None

In [25]:
for borough in df_toronto['Borough'].unique():
    print(borough)
    df_brgh=df_toronto[df_toronto['Borough']==borough]
    if df_venues is None:
        df_venues=getNearbyVenues(names=df_brgh['Neighbourhood'],
                                  latitudes=df_brgh['Latitude'],
                                 longitudes=df_brgh['Longitude'])
    else:
        df_venues=pd.concat([df_venues,getNearbyVenues(names=df_brgh['Neighbourhood'],
                                  latitudes=df_brgh['Latitude'],
                                 longitudes=df_brgh['Longitude'])])

Downtown Toronto
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
East Toronto
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
West Toronto
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Central Toronto
Lawrence

In [26]:
df_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
106,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,RBC Royal Bank,43.688058,-79.394478,Bank
107,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,Pizzaiolo,43.687991,-79.394634,Pizza Place
108,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,TTC Stop #,43.685826,-79.404981,Light Rail Station
109,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,Kiva's,43.687984,-79.394715,Bagel Shop


All the venues for each neighborhood.

In [27]:
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [29]:
print("No of unique categories of all returned values",len(df_venues['Venue Category'].unique()))

No of unique categories of all returned values 231


In [33]:
df_onehot=pd.get_dummies(df_venues[['Venue Category']])
df_onehot['Neighborhood']=df_venues['Neighborhood']

fixed_columns=[df_onehot.columns[-1]]+list(df_onehot.columns[:-1])
df_onehot=df_onehot[fixed_columns]
df_onehot.head()

,Neighborhood,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,...,Venue Category_Tibetan Restaurant,Venue Category_Toy / Game Store,Venue Category_Trail,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df_onehot.shape

(1603, 232)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
df_grouped=df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,...,Venue Category_Tibetan Restaurant,Venue Category_Toy / Game Store,Venue Category_Trail,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.060606,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
df_grouped.shape

(39, 232)

#### Each neighborhood along with the top 5 most common venues

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Venue Category_Coffee Shop,Venue Category_Cocktail Bar,Venue Category_Bakery,Venue Category_Cheese Shop,Venue Category_Restaurant,Venue Category_Seafood Restaurant,Venue Category_Beer Bar,Venue Category_Farmers Market,Venue Category_Gourmet Shop,Venue Category_Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Nightclub,Venue Category_Coffee Shop,Venue Category_Bakery,Venue Category_Climbing Gym,Venue Category_Burrito Place,Venue Category_Stadium,Venue Category_Restaurant,Venue Category_Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Venue Category_Light Rail Station,Venue Category_Pizza Place,Venue Category_Auto Workshop,Venue Category_Comic Shop,Venue Category_Restaurant,Venue Category_Burrito Place,Venue Category_Brewery,Venue Category_Skate Park,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Venue Category_Airport Service,Venue Category_Airport Lounge,Venue Category_Airport Terminal,Venue Category_Airport,Venue Category_Bar,Venue Category_Coffee Shop,Venue Category_Rental Car Location,Venue Category_Sculpture Garden,Venue Category_Boat or Ferry,Venue Category_Boutique
4,Central Bay Street,Venue Category_Coffee Shop,Venue Category_Italian Restaurant,Venue Category_Sandwich Place,Venue Category_Café,Venue Category_Bubble Tea Shop,Venue Category_Salad Place,Venue Category_Burger Joint,Venue Category_Diner,Venue Category_Indian Restaurant,Venue Category_Donut Shop


### Cluster Neighborhoods

In [40]:
from sklearn.cluster import KMeans
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 0, 4, 4, 4, 4])

In [41]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_toronto

df_merged =df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Bakery,Venue Category_Pub,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Theater,Venue Category_Yoga Studio,Venue Category_Spa,Venue Category_Dessert Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Venue Category_Coffee Shop,Venue Category_Sushi Restaurant,Venue Category_Yoga Studio,Venue Category_Fast Food Restaurant,Venue Category_Restaurant,Venue Category_Portuguese Restaurant,Venue Category_Park,Venue Category_Nightclub,Venue Category_Music Venue,Venue Category_Mexican Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Venue Category_Clothing Store,Venue Category_Coffee Shop,Venue Category_Hotel,Venue Category_Middle Eastern Restaurant,Venue Category_Bubble Tea Shop,Venue Category_Café,Venue Category_Japanese Restaurant,Venue Category_Cosmetics Shop,Venue Category_Pizza Place,Venue Category_Lingerie Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Gastropub,Venue Category_Cocktail Bar,Venue Category_American Restaurant,Venue Category_Gym,Venue Category_Italian Restaurant,Venue Category_Restaurant,Venue Category_Clothing Store,Venue Category_Moroccan Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Venue Category_Pub,Venue Category_Neighborhood,Venue Category_Coffee Shop,Venue Category_Health Food Store,Venue Category_Trail,Venue Category_Yoga Studio,Venue Category_Dessert Shop,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center


In [42]:
# import libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters